## 06. 반복기와 생성기

### 06.1 반복기 이해하기

#### 실습프로젝트생성
```bash
npm init -y
npm i -D typescript ts-node @types/node
mkdir srd
tsc --init
```
##### tsconfig.json
* `downlevelIteration": true`확인

```json
{
  "compilerOptions": {
    "module": "commonjs",
    "esModuleInterop": true,
    "target": "es2015",      
    "moduleResolution": "node",      
    "outDir": "dist",
    "baseUrl": ".",
    "sourceMap": true,
    "downlevelIteration": true,
    "noImplicitAny": false,
    "paths": { "*": ["node_modules/*"] }
  },
  "include": ["src/**/*"]
}
```

#### 반복기와 반복기 제공자

##### 1) src/createRangeIterable.ts
* createRangeIterable함수는 next()메서드가 있는 객체를 리턴

```ts
export const createRangeIterable = (from: number, to: number) => {
  let currentValue = from
  return {
    next() {
      const value = currentValue < to ? currentValue++ : undefined
      const done = value == undefined
      return { value, done }
    }
  }
}
```
##### 2) src/createRangeIterable-test.ts
* createRangeIterable함수를 호출해 반복기를 얻고 iterator변수에 저장

```ts
import { createRangeIterable } from './createRangeIterable'
const iterator = createRangeIterable(1, 3 + 1)
while (true) {
  const { value, done } = iterator.next()
  if (done) break
  console.log(value) // 1 2 3
}

```
###### createRangeIterable-test.ts 실행
```bash
ts-node ./src/createRangeIterable-test.ts 
```

#### 반복기는 왜 필요한가?
* 반복기 제공자는 특정 범위의 값을 한꺼번에 생성해서 배열에 담지않고 필요시마다 값을 생성

```ts
// 반복기가 아닌 예제로 사용한 range함수는 값이 필요시점보다 사전에 값을 미리 생성
export const range = (from, to) => from < to ? [from, ...range(from + 1, to)] : []
```

#### for...of구문과 [Symbol.iterator] 메서드
* 예제의 range함수는 for...of구문에 사용할 수 있다.

```ts
import {range} from './range'
for(let value of rang(1, 3+1)) 
    console.log(value); // 1, 2, 3
```
* 하지만 createRangeIterable함수는 for...of구문에 사용하면 `[Symbol.iterator]()메서드가 없다! 오류발생`

###### src/createRangeIterable-forOf.ts

```ts
import { createRangeIterable } from './createRangeIterable'
const iterable = createRangeIterable(1, 3 + 1)
// 에러발생
// for(let value of iterable) 
//    console.log(value)
```
* 이 오류는 다음의  RangeIterable.ts처럼 클래스로 구현해야 한다.

###### src/RangeIterable.ts

```ts
export class RangeIterable {
  constructor(public from: number, public to: number) {}
  [Symbol.iterator]() {
    const that = this
    let currentValue = that.from
    return {
      next() {
        const value = currentValue < that.to ? currentValue++ : undefined
        const done = value == undefined
        return { value, done }
      }
    }
  }
}
```
###### src/RangeIterable-test.ts

```ts
import { RangeIterable } from './RangeIterable'
const iterable = new RangeIterable(1, 3 + 1)

for (let value of iterable) {
  console.log(value)
}
```

>$Symbol.iterator$
>* Symbol.iterator는 ECMAScript 2015(ES6)에서 도입된 심볼로, 객체의 기본 이터레이터를 정의하는 데 사용
>* 이는 객체가 이터러블 프로토콜을 준수하도록 하여 for...of 루프, 전개 연산자 등과 함께 사용할 수 있게 한다. 
>* `이터러블과 이터레이터 프로토콜`
>  * <b>이터러블(Iterable)</b>: 
>    * Symbol.iterator 메서드를 가지고 있으며, 
>    * 이 메서드는 `이터레이터 객체를 반환`합니다. 
>    * 배열, 문자열, Map, Set 등은 기본적으로 이터러블입니다. 
>  * <b>이터레이터(Iterator)</b>: 
>    * next() 메서드를 구현하고 있으며, 
>    * 이 메서드는 `{ value: any, done: boolean } 형태의 객체를 반환`합니다.
>* 사용 예시
>  
```js
// 배열은 기본적으로 이터러블이므로, Symbol.iterator 메서드를 통해 이터레이터를 얻을 수 있습니다:
const array = [1, 2, 3];
const iterator = array[Symbol.iterator]();>
console.log(iterator.next()); // { value: 1, done: false }
console.log(iterator.next()); // { value: 2, done: false }
console.log(iterator.next()); // { value: 3, done: false }
console.log(iterator.next()); // { value: undefined, done: true }
```
```js
//  * 사용자 정의 객체에 Symbol.iterator를 구현하여 이터러블로 만들 수도 있습니다:
const iterableObject = {
  data: [10, 20, 30],
  [Symbol.iterator]() {
    let index = 0;
    const data = this.data;
    return {
      next() {
        if (index < data.length) {
          return { value: data[index++], done: false };
        } else {
          return { value: undefined, done: true };
        }
      }
    };
  }
};
for (const value of iterableObject) {
  console.log(value); // 10, 20, 30 출력
}
// 이처럼 Symbol.iterator를 활용하면 객체를 이터러블로 만들어 다양한 반복 작업에 활용할 수 있다.
```

#### Iterable<T>와 Iterator<T> 인터페이스
* 타입스크립트는 반복기 제공자에 Iterable<T>와 Iterator<T> 인터페이스를 사용할 수 있다.
* Iterable<T>는 자신을 구현하는 클래스가 Symbol.iterator메서드를 제공한다.
* `class 구현클래스 implements Iterable<생성할 값의 타입> {}`
* 또한,  `[Symbol.iterator](): Iterator<생성할 값의 타입> {}`

###### src/StringIterable.ts
```ts
export class StringIterable implements Iterable<string> {
  constructor(private strings: string[] = [], private currentIndex: number = 0) {}
  [Symbol.iterator](): Iterator<string> {
    const that = this
    let currentIndex = that.currentIndex,
      length = that.strings.length
    const iterator: Iterator<string> = {
      next(): { value: string; done: boolean } {
        const value = currentIndex < length ? that.strings[currentIndex++] : undefined
        const done = value == undefined
        return { value, done }
      }
    }
    return iterator
  }
}                                      
```
###### src/StringIterable-test.ts
                                           
```ts
import { StringIterable } from './StringIterable'
for (let value of new StringIterable(['hello', 'world', '!'])) console.log(value)
```

###### test 실행
```bash
ts-node src/StringIterable-test.ts
```

### 06.2  생성기
* ESNext 자바스크립트와 타입스크립트는 `yield라는 키워드를 제공`
* yield는 마치 return처럼 값을 반환한다.
* `yield는 반드시 function* 키워드를 사용한 함수만 호출 가능`
* 이렇게 `function* 키워드로 만드는 함수를 생성기(generator)`라고 한다.


```ts

```

```ts

```

```ts

```

```ts

```

```ts

```

```ts

```
```ts

```

```ts

```
```ts

```

```ts

```